Thư viện cần có

In [38]:
!pip install datasets transformers[sentencepiece] sacrebleu sacremoses


In [ ]:
import os
import sys
import transformers
import tensorflow as tf
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import AdamWeightDecay
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

In [ ]:
model_checkpoint = "Helsinki-NLP/opus-mt-ja-vi"

In [10]:
raw_dataset_train = load_dataset("dichmau/ja_vi_translation",split='train[:80%]')
raw_dataset_validation = load_dataset("dichmau/ja_vi_translation",split='train[2289341:2575507]')
raw_dataset_test = load_dataset("dichmau/ja_vi_translation",split='train[2575508:2861674]')
raw_datasets = DatasetDict({
    'train': raw_dataset_train,
    'test': raw_dataset_test,
    'valid': raw_dataset_validation})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [12]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 2289341
    })
    test: Dataset({
        features: ['input', 'output'],
        num_rows: 286166
    })
    valid: Dataset({
        features: ['input', 'output'],
        num_rows: 286166
    })
})

In [14]:
raw_datasets['train'][1]

{'input': '美しい白亜の壁も、優美な尖塔も、豪華な家具や内装も燃えていた。',
 'output': 'Ngọn lửa vây bọc lấy tòa biệt thự, đốt cháy cả bức tường đá trắng tuyệt đẹp, cả ngọn tháp mỹ lệ tao nhã, cả những đồ nội thất và trang trí sang trọng trong nhà.'}

In [15]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/840k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/763k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.68M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [16]:
tokenizer('私の名前はアンドレイです')

{'input_ids': [28759, 24176, 1532, 211, 0], 'attention_mask': [1, 1, 1, 1, 1]}

In [17]:
tokenizer(["私たちは数日前に日本に到着したばかりです。"])

{'input_ids': [[1910, 14774, 2281, 13910, 21432, 582, 24920, 8, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [18]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["chúng tôi vừa mới tới nhật bản được ít hôm."]))

{'input_ids': [[27, 22, 867, 262, 321, 2621, 335, 28, 775, 1211, 2, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [19]:
max_input_length = 128
max_target_length = 128

source_lang = "ja"
target_lang = "vi"



def preprocess_function(examples):
    inputs = [ex for ex in examples["input"]]
    targets = [ex for ex in examples["output"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [21]:
preprocess_function(raw_datasets["train"][:2])

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


{'input_ids': [[12, 24781, 63, 14226, 27889, 3043, 8, 0], [3932, 6744, 24766, 14746, 249, 55, 15578, 8813, 261, 39089, 11858, 249, 55, 38805, 28612, 445, 4178, 9139, 249, 19097, 3043, 8, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[2076, 6394, 19795, 325, 111, 4253, 21532, 428, 305, 2, 0], [22306, 312, 1547, 4472, 6577, 438, 1683, 485, 19795, 3, 3235, 3398, 115, 1388, 1151, 1435, 2370, 707, 793, 3, 115, 2984, 4846, 6659, 804, 1059, 6145, 3, 115, 26, 154, 1825, 1402, 18, 384, 691, 1336, 242, 34, 159, 2, 0]]}

In [23]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/2289341 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/286166 [00:00<?, ? examples/s]

Map:   0%|          | 0/286166 [00:00<?, ? examples/s]

In [24]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2289341
    })
    test: Dataset({
        features: ['input', 'output', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 286166
    })
    valid: Dataset({
        features: ['input', 'output', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 286166
    })
})

In [25]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

tf_model.h5:   0%|          | 0.00/312M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-ja-vi.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [26]:
batch_size = 16
learning_rate = 2e-5
weight_decay = 0.01
num_train_epochs = 1

In [27]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [28]:
generation_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=128)

In [29]:
train_dataset = model.prepare_tf_dataset(
    tokenized_datasets["test"],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)

In [31]:
validation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["valid"],
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator,
)

In [32]:
generation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["valid"],
    batch_size=8,
    shuffle=False,
    collate_fn=generation_data_collator,
)

In [33]:
optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)

In [ ]:
model.save_pretrained("tf_model/")

In [39]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = TFAutoModelForSeq2SeqLM.from_pretrained("tf_model/")

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at tf_model/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [40]:
input_text  = "又パラメーター見直し必要があれば、ITのほうで手配宜しくお願い致します。"

tokenized = tokenizer([input_text], return_tensors='np')
out = model.generate(**tokenized, max_length=128)
print(out)

tf.Tensor(
[[64501   318    80   177    66   957  1047    83   615   165     3   465
    370   187  1010   740   371   135   130   893     2     0 64501 64501
  64501]], shape=(1, 25), dtype=int32)


In [41]:
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out[0], skip_special_tokens=True))

Nếu bạn cần phải kiểm tra lại tham số, xin vui lòng cung cấp thông tin trên mạng.
